In [31]:
import os
from os.path import join as opj
import numpy as np
import random

import open3d as o3d
from open3d.web_visualizer import draw
import matplotlib.pyplot as plt
import trimesh

from utils import *
from dataset import *

In [32]:
import pickle as pkl
data_root = 'data_root/3DGraspAff/'
new_aff_path = opj(data_root, 'scaled_anntd_remapped_full_shape_test_data.pkl')
with open(new_aff_path, 'rb') as f:
    new_aff_dataset = pkl.load(f)
print("Loaded dataset with {} objects".format(new_aff_dataset['num_objects']))
print("Num of semantic classes: ", len(new_aff_dataset["semantic_classes"]))
print("Semantic classes: ", new_aff_dataset["semantic_classes"])
print("Num of affordances: ", len(new_aff_dataset["affordance_classes"]))
print("All affordances: ", new_aff_dataset["affordance_classes"])

Loaded dataset with 152 objects
Num of semantic classes:  7
Semantic classes:  ['Knife', 'Bag', 'Bottle', 'Scissors', 'Mug', 'Earphone', 'Hat']
Num of affordances:  7
All affordances:  ['handover', 'cut', 'stab', 'lift', 'wrap', 'pour', 'wear']


In [33]:
## pick a random datapoint
# set_random_seed(0)
# pick a random sem class
sem_class = random.choice(new_aff_dataset['semantic_classes']) # 'Scissors'
# pick a random shape id
shape_id = random.choice(list(new_aff_dataset['data'][sem_class].keys())) # '297c4f8e6d89d0b33d35434498261b'
datapoint = new_aff_dataset['data'][sem_class][shape_id]

In [34]:
print("Shape id ", shape_id, "\nSemantic class: ", sem_class)
pcl = datapoint['coordinates']
# Viz pcl with o3d
pcd = o3d.geometry.PointCloud()
pcd.points = o3d.utility.Vector3dVector(pcl)
# color black
pcd.paint_uniform_color([0, 0, 0])
# viz in jupyter nb
print("Drawing pcd with", len(pcl), "points")
draw(pcd)

Shape id  cbc1512b28e9ed382bef451de0ed6949 
Semantic class:  Bag
Drawing pcd with 2048 points
[Open3D INFO] Window window_20 created.


WebVisualizer(window_uid='window_20')

In [35]:
# get pcls of all labels
labels_list = []
scores_list = []
for key in datapoint['labels'].keys():
    labels = datapoint['labels'][key]
    scores = labels.sum()
    print("Label: ", key, " Score: ", scores)
    labels_list.append(labels)
    scores_list.append(scores)

    labelled_pcd = o3d.geometry.PointCloud()
    labelled_pcd.points = o3d.utility.Vector3dVector(pcl)
    labelled_pcd.paint_uniform_color([0, 0, 0])
    np.asarray(labelled_pcd.colors)[labels.squeeze() > 0] = np.random.uniform(0, 1, 3)
    print("Label 0: ", key)
    draw([labelled_pcd])

Label:  lift  Score:  34.699223
Label 0:  lift
[Open3D INFO] Window window_21 created.


WebVisualizer(window_uid='window_21')

Label:  handover  Score:  378.5432
Label 0:  handover
[Open3D INFO] Window window_22 created.


WebVisualizer(window_uid='window_22')

In [36]:
# Load and visualize the shapenet object
from utils.partnet_utils import class_to_shapenet_dir, shapeid_to_partnet_dir
# make keys values and values keys
SHAPENET_PATH = '/home/sjauhri/IAS_WS/ShapeNetCore.v2'
PARTNET_PATH  = '/home/sjauhri/IAS_WS/PartNet/data_v0'
rescale_shapenet_pcl = False

try :
    obj_path = opj(SHAPENET_PATH, class_to_shapenet_dir[sem_class.lower()],
                    shape_id, 'models', 'model_normalized.obj')
    # Load with trimesh
    # obj_mesh = trimesh.load(obj_path)
    # obj_mesh.show()
    # load with open3d
    obj_mesh = o3d.io.read_triangle_mesh(obj_path)
    ## Re-center and re-scale the corresponding point cloud to match the original shape
    orig_points = obj_mesh.sample_points_uniformly(number_of_points=8000).points
    # get the center of the mesh
    orig_center = np.mean(orig_points, axis=0)
    # get the maximum extent of the mesh
    centered_points = orig_points - orig_center # center the points
    orig_max_dist = np.max(np.sqrt(np.sum(centered_points**2, axis=1)))

    # correct the pcl
    corrected_pcl = pcl.copy()
    if rescale_shapenet_pcl:
        # flip along the z axis??
        corrected_pcl[:,2] = -corrected_pcl[:,2]
        # # flip along the x axis??
        corrected_pcl[:,0] = -corrected_pcl[:,0]
        # scale the point cloud
        corrected_pcl *= orig_max_dist
        # recenter the point cloud
        corrected_pcl += orig_center
except:
    print("ShapeNet category not found, try PartNet?") 
    partnet_anno_id = datapoint['partnet_anno_id']
    meshes_path = opj(PARTNET_PATH, partnet_anno_id, 'objs')
    # loop through all meshes in the folder and add to open3d object
    obj_mesh = o3d.geometry.TriangleMesh()
    for mesh in os.listdir(meshes_path):
        # optional: only load files with 'original' in the name
        mesh_path = opj(meshes_path, mesh)
        mesh = o3d.io.read_triangle_mesh(mesh_path)
        obj_mesh += mesh
    corrected_pcl = pcl

# Viz with o3d/trimesh
corrected_pcd = o3d.geometry.PointCloud()
corrected_pcd.points = o3d.utility.Vector3dVector(corrected_pcl)
# color red
corrected_pcd.paint_uniform_color([1, 0, 0])
# draw([obj_mesh,corrected_pcl_pcd])
axes_mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.6)
draw([obj_mesh, corrected_pcd, axes_mesh])

[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Skipping non-triangle primitive geometry of type: 2
[Open3D INFO] Window window_23 created.


WebVisualizer(window_uid='window_23')

[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceServers
[Open3D INFO] [Called HTTP API (custom handshake)] /api/call
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] DataChannelObserver::OnStateChange label: ServerDataChannel, state: open, peerid: 0.6289179292104392
[Open3D INFO] DataChannelObserver::OnStateChange label: ClientDataChannel, state: open, peerid: 0.6289179292104392
[Open3D INFO] Sending init frames to window_23.
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/addIceCandidate
[Open3D INFO] [Called HTTP API (custom handshake)] /api/getIceCandidate


In [37]:
# # scale down the mesh to 10% of the original size
# new_mesh = o3d.geometry.TriangleMesh()
# new_mesh.vertices = o3d.utility.Vector3dVector(
#     np.asarray(obj_mesh.vertices) * np.array([0.2, 0.2, 0.2]) )
# new_mesh.triangles = o3d.utility.Vector3iVector(np.asarray(obj_mesh.triangles))
# # scale down the point cloud to 10% of the original size
# corrected_pcd.points = o3d.utility.Vector3dVector(
#     np.asarray(corrected_pcd.points) * np.array([0.2, 0.2, 0.2]) )
# draw([new_mesh,obj_mesh,corrected_pcd])